In [1]:
from multiprocessing.sharedctypes import Value
import networkx as nx
import numpy as np
import random
import pandas as pd

from tqdm import tqdm
from datetime import datetime
import pickle

# File imports
import data
from data import select_dataset, get_station_g, ingest_electricity_data, set_random_speed_columns, ingest_pems
from replicate_graph import layer_graph
from vehicle import Vehicle
from simulation import Simulation
from visualization import set_draw_attributes

## Ingest pems for the whole graph (all charging and parking stops)
### commented out because it only needs to be done once. Csv already saved.

In [3]:
# all_graph = ingest_pems("data/charging_parking_stations_lonlat.csv", "data/charging_parking_distances.csv")

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [34:35<00:00, 86.47s/it]


In [14]:
# #save the new csv with all the speeds (from pems ingest above), which will be used for the algorithm
# all_graph[1].to_csv("data/charging_parking_distances_pemsingested.csv")

## Run the algorithm

In [2]:
number_of_iterations = 5
kwh_per_km = 1.9

#scenario list: each list inside has three params : charging rate, kwh per km, and battery capacity.
scenario_list = [[45,1.9,215],[90,1.9,215],[45,0.95,215],[45,1.9,430],[90,0.95,430]] 

for scenario, s in zip(scenario_list, range(0,len(scenario_list))):
    
    print('------- SCENARIO '+str(s)+' -------')
    #initialize for wcctici
    simulation_length = 12
    battery_interval = 20
    kwh_per_km = scenario[1]
    battery_capacity = scenario[2]
    stations_path = "data/wcctci_stations-updated.csv"
    distances_path = "data/wcctci_coord_distances.csv"
    name = "scenario"+str(s)+"_wcctci" 
    
    for n in range(0, number_of_iterations):

        sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
        sim.add_demand_nodes()
        sim.run()

        ### Print metrics

        #print name of this simulation
        print(name, )
        print("station_utilization_disp_of_avg:  ", sim.metrics['station_utilization_disp_of_avg'])
        print("station_utilization_avg_of_disp:  ", sim.metrics['station_utilization_avg_of_disp'])
        print("electricity:  ", sim.metrics['electricity'])
        print("percent_delay:  ", sim.metrics['percent_delay'])
        print("hours_spent_in_queues:  ", sim.metrics['hours_spent_in_queues'])
        print("hours_spent_charging:  ", sim.metrics['hours_spent_charging'])
        print('____________________')


        ### get station utilization

        #get the station ids in order 
        current_stations = [int(j) for j in sim.station_g.nodes]

        #get the utilization rate of the stations
        utilization = sim.metrics['station_utilization']

        #get the station that has the highest utilization.
        h_st = current_stations[np.argmax(utilization)]

        #get the station that has the lowest utilization.
        l_st = current_stations[np.argmin(utilization)]


        ### Find the unused location that is closest to the highly utilized station

        #get the map of all parking areas and stations
        full_map_dist = pd.read_csv("data/charging_parking_distances_pemsingested.csv")
        full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

        #keep only the roads that start or end with the high utilization station
        h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

        #then we eliminate all of the stations that are already being used.
        using_st = current_stations.copy()
        using_st.remove(h_st)
        h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

        #then we remove the case where we go from h_st to h_st
        h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

        #find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

        new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
        new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

        #one of the two will be h_st and the other will be the new station location. 
        if new_origin == h_st:
            new_station = new_destination
        else:
            new_station = new_origin


        ### create the new maps with the locations and the distances of the stations that we will be using next

        new_stations_list = current_stations + [new_station]
        new_stations_list.remove(l_st)

        #now we reduce these to the new set of stations. All origins and destinations should be in this list.
        new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
        new_map_loc = full_map_loc[full_map_loc.loc[:, 'OID_'].isin(new_stations_list)]

        #we add the charging rates and capacity- assuming that they are the same as the others (set to 45 and 8)
        #TO DO: check this
        new_map_loc['charging_rate'] = scenario[0]
        new_map_loc['physical_capacity'] = 8

        #we change the snapX and snapY columns to lon and lat 
        new_map_loc = new_map_loc.rename(columns = {'SnapX': 'longitude', 'SnapY': 'latitude'})

        #now we save them. 
        new_map_loc.to_csv("data/algorithm_"+str(n+1)+"_stations.csv") 
        new_map_dist.to_csv("data/algorithm_"+str(n+1)+"_distances.csv")


        #reset the variables for the next iteration
        stations_path = "data/algorithm_"+str(n+1)+"_stations.csv"
        distances_path = "data/algorithm_"+str(n+1)+"_distances.csv"
        name = "scenario"+str(s)+"_algorithm_"+str(n+1)
    
    

    #we run the simulation for the final alocation

    ### run the simulation

    sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
    sim.add_demand_nodes()
    sim.run()


    ### Print metrics

    #print name of this simulation
    print(name)
    print("station_utilization_disp_of_avg:  ", sim.metrics['station_utilization_disp_of_avg'])
    print("station_utilization_avg_of_disp:  ", sim.metrics['station_utilization_avg_of_disp'])
    print("electricity:  ", sim.metrics['electricity'])
    print("percent_delay:  ", sim.metrics['percent_delay'])
    print("hours_spent_in_queues:  ", sim.metrics['hours_spent_in_queues'])
    print("hours_spent_charging:  ", sim.metrics['hours_spent_charging'])

------- SCENARIO 0 -------


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:05<00:00,  5.48s/it]


scenario0_wcctci
station_utilization_disp_of_avg:   17.42682291666667
station_utilization_avg_of_disp:   17.4359375
electricity:   0
percent_delay:   (1.0, 1.081176268014861, 4.645865529389639, 1.3455363922829928, 0.7322992289386983)
hours_spent_in_queues:   (0.0, 0.0, 7.0, 0.34711864406779663, 1.086207912874594)
hours_spent_charging:   (0, 0.0, 24, 2.4101694915254237, 6.128944801354659)
____________________


<ipython-input-2-15e4afd591bb>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['charging_rate'] = scenario[0]
<ipython-input-2-15e4afd591bb>:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['physical_capacity'] = 8
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:21<00:00,  6.76s/it]


scenario0_algorithm_1
station_utilization_disp_of_avg:   14.521354166666669
station_utilization_avg_of_disp:   14.551822916666667
electricity:   0
percent_delay:   (1.3259882524244162, 1.5730789851367193, 4.948856749458465, 1.6879051831443552, 0.5947427810779189)
hours_spent_in_queues:   (0.0, 0.0, 5.6, 0.3889273356401383, 1.048387404745609)
hours_spent_charging:   (0, 0.0, 24, 1.2802768166089966, 3.8077945818218457)
____________________


...

scenario0_algorithm_3
station_utilization_disp_of_avg:   14.888281249999999
station_utilization_avg_of_disp:   14.890104166666667
electricity:   0
percent_delay:   (1.2166331243308481, 1.5730789851367193, 3.90977064103651, 1.5362665075521957, 0.5399678917840401)
hours_spent_in_queues:   (0.0, 0.0, 5.2, 0.1675090252707581, 0.6921623659902882)
hours_spent_charging:   (0, 0.0, 24, 1.1913357400722022, 4.507057925075535)
____________________


<ipython-input-2-15e4afd591bb>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['charging_rate'] = scenario[0]
<ipython-input-2-15e4afd591bb>:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['physical_capacity'] = 8
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:31<00:00,  2.59s/it]


scenario0_algorithm_4
station_utilization_disp_of_avg:   14.482812500000001
station_utilization_avg_of_disp:   14.48671875
electricity:   0
percent_delay:   (1.2166331243308481, 1.5730789851367193, 4.5448684433802224, 1.5407161798197102, 0.5785964365698943)
hours_spent_in_queues:   (0.0, 0.0, 4.4, 0.2054421768707483, 0.796914313792414)
hours_spent_charging:   (0, 0.0, 24, 0.8945578231292517, 3.838460616773401)
____________________


<ipython-input-2-15e4afd591bb>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['charging_rate'] = scenario[0]
<ipython-input-2-15e4afd591bb>:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['physical_capacity'] = 8
  0%|                                                                                           | 0/12 [00:01<?, ?it/s]


NetworkXNoPath: No path between Long Beach and San Diego.

## Bellow this is  just code I used to think about/ build the algorithm above (Just ignore).
### I was thinking about just one iteration, starting from the wcctci pickle.

In [2]:
#open the pickle
with open('trials/wcctci_04_04_2022_12_15_28.pkl', 'rb') as inp:
    res = pickle.load(inp)

Get the highest and lowest utilized stations:

In [7]:
#get station utilization

#get the station ids in order 
current_stations = list(res.station_g.nodes)

#get the utilization rate of the stations
utilization = res.metrics['station_utilization']

#get the station that has the highest utilization.
h_st = current_stations[np.argmax(utilization)]

#get the station that has the lowest utilization.
l_st = current_stations[np.argmin(utilization)]

Find the unused location that is closest to the highly utilized station:

In [41]:
#get the map of all parking areas and stations
full_map_dist = pd.read_csv("data/charging_parking_distances.csv")

#keep only the roads that start or end with the high utilization station
h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

#then we eliminate all of the stations that are already being used.
using_st = current_stations.copy()
using_st.remove(h_st)
h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

#then we remove the case where we go from h_st to h_st
h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

#find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

#one of the two will be h_st and the other will be the new station location. 
if new_origin == h_st:
    new_station = new_destination
else:
    new_station = new_origin

Now we start to think about re-runing the simulation with this swap (taking out l_st and adding new_station). To do this, we need to create a new map with the locations and the distances of the stations that we will be using next.

In [ ]:
new_stations_list = current_stations + [new_station]

In [ ]:
#we have "full_map_dist" (distances map)
#and we need the locations map
full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

#now we reduce these to the new set of stations. All origins and destinations should be in this list.
new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
new_map_loc = full_map_loc[(full_map_loc.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_loc.loc[:, 'DestinationID'].isin(new_stations_list))]

In [ ]:
#now we save them. 
new_map_loc.to_csv("data/algorithm_"+str(1)+"_stations.csv") ########change the 1
new_map_dist.to_csv("data/algorithm_"+str(1)+"_distances.csv")

Now we can run the next simulation

In [ ]:
simulation_length = 12
battery_interval = 20
km_per_percent = 3.13

sim = Simulation("data/algorithm_"+str(1)+"_stations.csv", "data/algorithm_"+str(1)+"_distances.csv", simulation_length, battery_interval, km_per_percent)

# Anna

In [17]:
stations_path = "data/wcctci_stations-updated.csv"
distances_path = "data/wcctci_coord_distances.csv"
# stations_df, distances_df = select_dataset(stations_csv_path, distances_csv_path)
# station_g = get_station_g(stations_df, distances_df)
# battery_g = layer_graph(station_g, 20, km_per_percent= 1.9)
simulation_length = 24
battery_interval = 20
kwh_per_km = 1.9
battery_capacity = 215
name = 'hi'
sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
sim.add_demand_nodes()

In [18]:
sim.battery_g.nodes

NodeView(('1_0_in', '2_0_in', '3_0_in', '4_0_in', '5_0_in', '6_0_in', '7_0_in', '8_0_in', '9_0_in', '10_0_in', '11_0_in', '12_0_in', '13_0_in', '14_0_in', '15_0_in', '16_0_in', '17_0_in', '18_0_in', '19_0_in', '20_0_in', '21_0_in', '22_0_in', '23_0_in', '24_0_in', '25_0_in', '26_0_in', '27_0_in', '28_0_in', '29_0_in', '30_0_in', '31_0_in', '32_0_in', '33_0_in', '34_0_in', '35_0_in', '36_0_in', '37_0_in', '38_0_in', '39_0_in', '40_0_in', '41_0_in', '42_0_in', '43_0_in', '1_20_in', '2_20_in', '3_20_in', '4_20_in', '5_20_in', '6_20_in', '7_20_in', '8_20_in', '9_20_in', '10_20_in', '11_20_in', '12_20_in', '13_20_in', '14_20_in', '15_20_in', '16_20_in', '17_20_in', '18_20_in', '19_20_in', '20_20_in', '21_20_in', '22_20_in', '23_20_in', '24_20_in', '25_20_in', '26_20_in', '27_20_in', '28_20_in', '29_20_in', '30_20_in', '31_20_in', '32_20_in', '33_20_in', '34_20_in', '35_20_in', '36_20_in', '37_20_in', '38_20_in', '39_20_in', '40_20_in', '41_20_in', '42_20_in', '43_20_in', '1_40_in', '2_40_in

In [22]:
nx.shortest_path(sim.battery_g, "Long Beach", "Reno")

['Long Beach',
 '9',
 '9_100_in',
 '9_100_out',
 '10_0_in',
 '10_100_out',
 '2_0_in',
 '2_80_out',
 '31_0_in',
 '31_80_out',
 '32_0_in',
 '32_80_out',
 '33_0_in',
 '33_80_out',
 '34_0_in',
 '34_80_out',
 '35_0_in',
 '35_100_out',
 '5_0_in',
 '5_100_out',
 '37_0_in',
 '37_80_out',
 '38_20_in',
 '38_20_out',
 '38',
 'Reno']